### first attempt with incomplete dataset
- unusable due to low performance

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from classes import Document, Annotation

In [3]:
import spacy
from spacy import displacy
from spacy.tokens import DocBin

In [3]:
data = pd.read_json('labels_new.json')
data.head()

,documents
0,"{'annotations': [{'end': 13, 'id': 1, 'label':..."
1,"{'annotations': [{'end': 18, 'id': 1, 'label':..."
2,"{'annotations': [{'end': 16, 'id': 1, 'label':..."
3,"{'annotations': [{'end': 22, 'id': 1, 'label':..."
4,"{'annotations': [{'end': 36, 'id': 1, 'label':..."


In [4]:
data['documents'][0]['annotations']

[{'end': 13, 'id': 1, 'label': 'jobTitle', 'start': 3},
 {'end': 31, 'id': 2, 'label': 'jobSkill', 'start': 13},
 {'end': 501, 'id': 3, 'label': 'jobSkill', 'start': 494},
 {'end': 1023, 'id': 4, 'label': 'jobSkill', 'start': 1014},
 {'end': 1110, 'id': 5, 'label': 'jobSkill', 'start': 1084},
 {'end': 1136, 'id': 6, 'label': 'jobSkill', 'start': 1131},
 {'end': 1334, 'id': 7, 'label': 'jobTitle', 'start': 1317},
 {'end': 1542, 'id': 8, 'label': 'softSkill', 'start': 1530},
 {'end': 3350, 'id': 9, 'label': 'jobTitle', 'start': 3325},
 {'end': 5508, 'id': 10, 'label': 'jobSkill', 'start': 5498},
 {'end': 5536, 'id': 11, 'label': 'jobSkill', 'start': 5528},
 {'end': 5540, 'id': 12, 'label': 'jobSkill', 'start': 5538},
 {'end': 5547, 'id': 13, 'label': 'jobSkill', 'start': 5542},
 {'end': 5642, 'id': 14, 'label': 'jobSkill', 'start': 5632},
 {'end': 5699, 'id': 15, 'label': 'jobSkill', 'start': 5685},
 {'end': 5760, 'id': 16, 'label': 'jobSkill', 'start': 5743},
 {'end': 5779, 'id': 17, 'l

In [5]:
for document in data['documents']:
    del document['id']
    for ann in document['annotations']:
        del ann['id']

In [6]:
data['documents'][0]

{'annotations': [{'end': 13, 'label': 'jobTitle', 'start': 3},
  {'end': 31, 'label': 'jobSkill', 'start': 13},
  {'end': 501, 'label': 'jobSkill', 'start': 494},
  {'end': 1023, 'label': 'jobSkill', 'start': 1014},
  {'end': 1110, 'label': 'jobSkill', 'start': 1084},
  {'end': 1136, 'label': 'jobSkill', 'start': 1131},
  {'end': 1334, 'label': 'jobTitle', 'start': 1317},
  {'end': 1542, 'label': 'softSkill', 'start': 1530},
  {'end': 3350, 'label': 'jobTitle', 'start': 3325},
  {'end': 5508, 'label': 'jobSkill', 'start': 5498},
  {'end': 5536, 'label': 'jobSkill', 'start': 5528},
  {'end': 5540, 'label': 'jobSkill', 'start': 5538},
  {'end': 5547, 'label': 'jobSkill', 'start': 5542},
  {'end': 5642, 'label': 'jobSkill', 'start': 5632},
  {'end': 5699, 'label': 'jobSkill', 'start': 5685},
  {'end': 5760, 'label': 'jobSkill', 'start': 5743},
  {'end': 5779, 'label': 'jobSkill', 'start': 5762},
  {'end': 5857, 'label': 'jobSkill', 'start': 5851},
  {'end': 5902, 'label': 'softSkill', 'st

In [31]:
training_data = []

for document in data['documents']:
    if len(document['content']) < 1:
        print(f'missing contents', {document['content']},{document['annotations'][0]['label']})
        continue
    else:
        text = document['content']
        entities = document['annotations']
        training_data.append({'text':text,'entities':entities})

missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}
missing contents {''} {'jobTitle'}


In [21]:
nlp = spacy.blank("en") 
doc_bin = DocBin()

In [ ]:
doc_count = len(training_data)
for d in training_data:
    label_count = len(d['entities'])
    print(label_count)

In [41]:
nonecount = 0

for d in tqdm(training_data):
    text = d['text']
    labels = d['entities']
    doc = nlp.make_doc(text)
    entities = []

    for l in labels:
        start = l['start']
        if text[start] == ' ':
            start +=1
        end = l['end']
        if text[end] == ' ':
            end -=1
        label = l['label']

        span = doc.char_span(start_idx=start, end_idx=end,label=label,alignment_mode='expand')
        if span == None:
            nonecount += 1
            continue
        entities.append(span)
    
print(nonecount)

100%|██████████| 185/185 [00:00<00:00, 227.52it/s]

0


In [42]:
doc.set_ents(entities)
doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

In [43]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [44]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    220.07    0.00    0.00    0.00    0.00
  0     200       2658.09  12968.38   45.06   47.11   43.18    0.45
  0     400        462.55   3361.68   60.90   56.00   66.76    0.61
  1     600        381.10   2917.59   75.93   77.86   74.09    0.76
  1     800        439.43   2299.82   79.84   75.40   84.83    0.80
  2    1000        474.82   1850.35   73.80   88.68   63.20    0.74
  2    1200        583.66   1890.21   84.11   83.40   84.83    0.84
  3    1400        576.72   1677.59   87.06   87.54   86.59    0.87
  3    1600        603.24   1383.61 

In [1]:
nlp_trained = spacy.load("model-best")

NameError: name 'spacy' is not defined

In [ ]:
with open('test.txt', 'r') as f:
    text = f.read()
doc = text


In [47]:
from IPython.display import HTML, display
html = displacy.render(doc, style="ent", jupyter=False)
display(HTML(f'<span class="tex2jax_ignore">{html}</span>'))

Data generation operation failed
 error details: AOAI resource authorization failed:
  Response: Forbidden, message: 
  {"error":
  {"code":"AuthorizationFailed",
  "message":
    "The client '007a9b5a-127f-4ae3-956a-6e0f29ec89ae' with object id '007a9b5a-127f-4ae3-956a-6e0f29ec89ae' does not have authorization to perform action 'Microsoft.CognitiveServices/accounts/listKeys/action' over scope '/subscriptions/aab34b8c-b2d8-42f1-af9b-72404d4d2e69/resourceGroups/cvbutbetter/providers/Microsoft.CognitiveServices/accounts/labellm' or the scope is invalid. If access was recently granted, please refresh your credentials."}} 